In [26]:
#Import necessary modules
import impact as                             impt
import impact.plotting as                    implot
from impact.parsers import Parser as parser
import numpy as np
#SETTINGS FOR IMPACT. Change settings to alter calculations as required
impt.settings.perform_curve_fit = False
impt.settings.fit_type = 'gompertz'
impt.settings.outlier_cleaning_flag = True
impt.settings.max_fraction_replicates_to_remove = 0.3
impt.settings.std_deviation_cutoff=0.1
impt.settings.verbose = False
impt.settings.live_calculations = False
impt.settings.use_filtered_data = True
impt.settings.death_phase_hyperparameter = 1
import plotly as py
import sys
if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML

    # This is required for correct mathjax (latex) and documentation rendering
    HTML(
        """
        <script>
            var waitForPlotly = setInterval( function() {
                if( typeof(window.Plotly) !== "undefined" ){
                    MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
                    MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
                    clearInterval(waitForPlotly);
                }}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

from plotly import tools
import plotly.graph_objs as go
import colorlover as cl
import math
import plotly.io as pio
pio.templates.default = "plotly_white"
import plotly.colors
from plotly.subplots import make_subplots
import pandas as pd

colors = {'blues':['rgb(164,199,228)','rgb(119,171,214)','rgb(73,143,201)','rgb(49,113,165)','rgb(36,82,120)','rgb(23,52,76)'],
          'greens':['rgb(172,220,173)','rgb(130,203,133)','rgb(89,185,92)','rgb(63,152,66)','rgb(46,110,48)','rgb(28,67,29)'],
         'reds':['rgb(230,174,176)','rgb(218,133,137)','rgb(205,92,98)','rgb(187,57,64)','rgb(146,45,50)','rgb(104,32,35)'],
          #['rgb'+str(color) for color in plotly.colors.n_colors((254,224,210), (165,15,21), 6)],
         'pinks':['rgb(215,181,210)','rgb(195,144,188)','rgb(175,107,165)','rgb(147,79,137)','rgb(110,59,102)','rgb(74,40,69)'],
         'browns':['rgb(216,197,184)','rgb(196,168,148)','rgb(176,139,112)','rgb(150,111,83)','rgb(115,85,63)','rgb(80, 59, 44)'],
         'oranges': ['rgb(255,206,146)','rgb(255,189,109)','rgb(255,173,73)','rgb(255,157,36)','rgb(255, 140, 0)','rgb(219,120,0)'],
          'cyans': ['rgb(177,222,219)','rgb(139,205,201)','rgb(100,189,183)','rgb(70,163,157)','rgb(53,124,120)','rgb(37,87,84)'],
          'yellows': ['rgb(244,239,186)','rgb(239,231,151)','rgb(233,224,116)','rgb(228,216,82)','rgb(223,208,47)','rgb(200,185,31)',],
         'blacks':['rgb(182,182,182)','rgb(146,146,146)','rgb(109,109,109)','rgb(73,73,73)','rgb(36,36,36)','rgb(0,0,0)']}

colors_bg =  {'blues':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blues']],
              'greens':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['greens']],
              'reds':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['reds']],
              'pinks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['pinks']],
              'browns':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['browns']],
              'oranges':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['oranges']],
              'cyans': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['cyans']],
              'yellows': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['yellows']],
              'blacks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blacks']]}

import pickle
with open('C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Experiments/Screening/Autofluorescence_Models/gfp_models.pickle', 'rb') as handle:
    gfp_autofl_models = pickle.load(handle)

In [2]:
def bar_plot_all(replicate_trials=None, analyte_of_interest='OD700', base_rep=None,
                 plot_start=0, plot_end=-1, plot_width=900, norm=True, od_norm=True):
    title = ''
    if analyte_of_interest == 'OD700':
        title += analyte_of_interest+' '
        attr = 'data_vector'
    else:
        if od_norm:
            title += 'OD Normalized '+analyte_of_interest+' '
            attr = 'od_normalized_data'
        else:
            title += analyte_of_interest+' '
            attr='data_vector'
            
    if base_rep:
        bg_fl = getattr(base_rep.avg.analyte_dict[analyte_of_interest], attr)[-1]
        bg_fl_err = getattr(base_rep.std.analyte_dict[analyte_of_interest], attr)[-1]
    else:
        bg_fl = 0
        bg_fl_err = 0
        
        
    if norm:
        title = title + 'scaled to the max value'
        
        
    max_ss = max([getattr(rep.avg.analyte_dict['GFP'], attr)[-1] - bg_fl for rep in replicate_trials])
    min_ss = min([getattr(rep.avg.analyte_dict['GFP'], attr)[-1] - bg_fl for rep in replicate_trials])
    
    
    #First separate by Degradation Tag - Observe effect of RBS
    trace_list = []
    for index,rbs_level in enumerate(['1','2','3','4','5']):
        reps=[]
        for rep in replicate_trials:
            if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
                ident = rep.trial_identifier.strain.name.replace('00','').split()
                if ident:
                    if ident[0][0] == rbs_level:
                        reps.append(rep)
        reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
        x = ['Degradation<br>Level 0', 'Degradation<br>Level 1', 'Degradation<br>Level 2', 'Degradation<br>Level 3']
        

        y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - bg_fl)
             for rep in reps]
        y_errs = [np.sqrt(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]**2 + bg_fl_err**2)
             for rep in reps]
        
        if norm:
            base = 0#min_ss
            scale = max_ss# - min_ss
        else:
            base=0
            scale=1
            
        y = [(value - base)/scale for value in y]
        y_errs = [value/scale for value in y_errs]
        
        if rbs_level=='1':
            showlegend=True
        else:
            showlegened=False
        if reps:
            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=colors['greens'][index], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(rbs_level),
                                 name='RBS Level: '+str(rbs_level)))

    strains = ['0000', 'Lac', 'Tet']
    temp_colors=[colors['blacks'][4], colors['oranges'][4], colors['pinks'][4]]
    for i,strain in enumerate(strains):
        x = ['Reference Strains']
        replicate = [rep for rep in replicate_trials if rep.trial_identifier.strain.name==strain]
        if replicate:
            replicate = replicate[0]
            y = [(getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1] - bg_fl)]
            y_errs = [np.sqrt(getattr(replicate.std.analyte_dict[analyte_of_interest], attr)[-1]**2 + bg_fl_err**2)]

            if norm:
                base = 0#min_ss
                scale = max_ss# - min_ss
            else:
                base=0
                scale=1

            y = [(value - base)/scale for value in y]
            y_errs = [value/scale for value in y_errs]

            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=temp_colors[i], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(strain),
                                 name=str(strain)))    
    width=plot_width
    fig1=go.Figure(data=trace_list)
    fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                      ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                      tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                      showgrid=False,zeroline=False)
    fig1.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                       title_font=dict(family='Myriad Pro',size=16, color='black'),
                       legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                       legend_font=dict(family='Myriad Pro', size=16, color='black'))
    fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
    fig1['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                                 title_font=dict(family='Myriad Pro', size=18, color='black'),
                                 showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                                 ticks='outside', ticklen=4, tickangle=0, type='log',
                                 tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                 showgrid=True,zeroline=False,side='left')
    
    trace_list = []
    for index,deg_level in enumerate(['0','1','2','3']):
        reps=[]
        for rep in replicate_trials:
            if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
                ident = rep.trial_identifier.strain.name.replace('00','').split()
                if ident:
                    if ident[0][1] == deg_level:
                        reps.append(rep)
        reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
        x = ['RBS<br>Level 1', 'RBS<br>Level 2', 'RBS<br>Level 3', 'RBS<br>Level 4', 'RBS<br>Level 5']
        

        y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - bg_fl)
             for rep in reps]
        y_errs = [np.sqrt(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]**2 + bg_fl_err**2)
             for rep in reps]
        
        if norm:
            base = 0#min_ss
            scale = max_ss# - min_ss
        else:
            base=0
            scale=1
            
        y = [(value - base)/scale for value in y]
        y_errs = [value/scale for value in y_errs]
        if deg_level=='0':
            showlegend=True
        else:
            showlegened=False
        if reps:
            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=colors['reds'][1:][index], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(deg_level),
                                 name='Degradation Level: '+str(deg_level)))
        
    width=plot_width
    fig2=go.Figure(data=trace_list)
    fig2.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                      ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                      tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                      showgrid=False,zeroline=False)
    fig2.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                        title_font=dict(family='Myriad Pro',size=18, color='black'),
                      legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                      legend_font=dict(family='Myriad Pro', size=16, color='black'))
    fig2['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')

    fig2['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                                 title_font=dict(family='Myriad Pro', size=18, color='black'),
                                 showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                                 ticks='outside', ticklen=4, tickangle=0, type='log',
                                 tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                 showgrid=True,zeroline=False,side='left')
    
    
    
    
    

    return fig1, fig2

In [20]:
data_format='tecan_OD_GFP_mCherry'
plate_type = '96 Wells'
file_name = '24Jan2021_Day1of2_pTSFusXX00_EndPoint.xlsx'
expt_atc = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
expt_atc.calculate()
autofl_rep = [rep for rep in expt_atc.replicate_trials if rep.trial_identifier.strain.name=='Lac'][0]

# gfp_model = gfp_autofl_models['film_spark'][90]
for replicate in expt_atc.replicate_trials:
    if replicate.trial_identifier.strain.name.lower() != 'blank':
        for single_trial in replicate.single_trials:
            single_trial.analyte_dict['GFP'].pd_series.update(single_trial.analyte_dict['GFP'].pd_series - 
                                                              autofl_rep.avg.analyte_dict['GFP'].pd_series)
            
expt_atc.calculate()


# data_format='tecan_OD_GFP_mCherry'
# plate_type = '96 Wells'
# file_name = 'pTSFus00XX_SparkEndPoint_24012021_D1.xlsx'
# expt_iptg = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
# expt_iptg.calculate()

# gfp_model = gfp_autofl_models['film_spark'][90]
# for replicate in expt_iptg.replicate_trials:
#     if replicate.trial_identifier.strain.name.lower() != 'blank':
#         for single_trial in replicate.single_trials:
#             single_trial.analyte_dict['GFP'].pd_series.update(single_trial.analyte_dict['GFP'].pd_series - 
#                                                               np.array([gfp_model(od_value) for od_value
#                                                                         in single_trial.analyte_dict['OD700'].pd_series.values]))
            
# expt_iptg.calculate()


Importing data from 24Jan2021_Day1of2_pTSFusXX00_EndPoint.xlsx...0.0s
Extracting timepoint data...Extracted time point data in 0.1s
Parsing time point list...Parsed 288 time points in 0.1s
Parsing analyte list...Parsed 96 single trials in 853.7ms
Parsing single trial list...Parsed 24 replicates in 0.1s
Analyzing data...Ran analysis in 1.0s

Analyzing data...Ran analysis in 0.9s



In [18]:
expt_atc.replicate_trials[0].single_trials[0].analyte_dict['GFP'].pd_series-autofl_rep.avg.analyte_dict['GFP'].data_vector

0.0    1523.5
dtype: float64

In [7]:
strain_lists = ['1000','2000','3000','4000','5000',#,'0000','Lac'],
               '1100','2100','3100','4100','5100',#,'0000','Lac'],
              '1200','2200','3200','4200','5200',#,'0000','Lac'],
              '1300','2300','3300','4300','5300']#,'0000','Lac']]


#gfp_norm_rep = [replicate for replicate in expt.stages['0-8'].replicate_trials if str(replicate.trial_identifier.strain.name)=='Lac'][0]



rep_list = [replicate for replicate in expt_atc.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strain_lists]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='GFP', plot_width=600, base_rep=None)
plot(fig1)
plot(fig2)

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='GFP', plot_width=600, norm=False,  base_rep=None)
plot(fig1)
fig1.write_image('fig1.svg', format='svg')
plot(fig2)

strains_2 = ['0000','Tet','Lac']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains_2]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='GFP', plot_width=300, norm=False, base_rep=gfp_norm_rep)
plot(fig1)


strains_2 = ['0000','Tet','Lac']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains_2]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='GFP', plot_width=300, norm=False, base_rep=None)
plot(fig1)


NameError: name 'expt' is not defined

In [23]:
strains = ['1000','2000','3000','4000','5000',#,'0000','Lac'],
               '1100','2100','3100','4100','5100',#,'0000','Lac'],
              '1200','2200','3200','4200','5200',#,'0000','Lac'],
              '1300','2300','3300','4300','5300']#,'0000','Lac']]
rep_list = [replicate for replicate in expt_atc.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
norm = True
analyte_of_interest = 'GFP'
title = ''
title += 'OD Normalized '+analyte_of_interest+' '
attr = 'data_vector'


max_ss = max([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
             for replicate in rep_list])
min_ss = min([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
             for replicate in rep_list])

if norm:
    title = title + 'scaled to the max value'

#First separate by Degradation Tag - Observe effect of RBS
trace_list = []
for index,rbs_level in enumerate(['1','2','3','4','5']):
    rbs_codes = {'1':'A', '2':'B', '3':'C', '4':'D', '5':'E'}
    reps=[]
    for rep in rep_list:
        if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
            ident = rep.trial_identifier.strain.name.replace('00','').split()
            if ident:
                if ident[0][0] == rbs_level:
                    reps.append(rep)
    reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
    x = ['No Tag', 'DAS Tag', 'DAS+2 Tag', 'LAA Tag']

    if norm:
        base = 0#min_ss
        scale = max_ss# - min_ss
    else:
        base=0
        scale=1

    y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
         for rep in reps]
    y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
         for rep in reps]

    if rbs_level=='1':
        showlegend=True
    else:
        showlegened=False
    if reps:
        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=py.colors.n_colors(py.colors.diverging.RdBu[-4], py.colors.diverging.RdBu[-2], 5, colortype='rgb')[index], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(rbs_level),
                             name='RBS "'+str(rbs_codes[rbs_level])+'"'))

width=700
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=450, width=625, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=18, color='black'),
                   legend_x=0.02, bargap=0.3, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=18, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')
fig1['layout']['yaxis'] = dict(title='Scaled Fluorescence (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1.25, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig1)
trace_list = []
for index,deg_level in enumerate(['0','1','2','3']):
    deg_codes = {'0': 'No Tag', '1':'DAS Tag', '2':'DAS+2 Tag', '3':'LAA Tag'}
    reps=[]
    for rep in rep_list:
        if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
            ident = rep.trial_identifier.strain.name.replace('00','').split()
            if ident:
                if ident[0][1] == deg_level:
                    reps.append(rep)
    reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
    x = ['RBS "A"', 'RBS "B"', 'RBS "C"', 'RBS "D"', 'RBS "E"']

    y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
         for rep in reps]
    y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
         for rep in reps]


    if deg_level=='0':
        showlegend=True
    else:
        showlegened=False
    if reps:
        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=py.colors.n_colors(colors['oranges'][1], colors['oranges'][-1], 4, colortype='rgb')[::-1][index], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(deg_level),
                             name=deg_codes[deg_level]))

fig2=go.Figure(data=trace_list)
fig2.update_xaxes(showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig2.update_layout(height=400, width=625, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=18, color='black'),
                   legend_x=0.02, bargap=0.3, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=18, color='black'))
fig2['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')
fig2['layout']['yaxis'] = dict(title='Scaled Fluorescence (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1.25, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig2)


# fig1.write_image("figXX00_RBS.svg", format='svg')
# fig2.write_image("figXX00_deg.svg", format='svg')





In [28]:
expt = expt_atc
strains = ['10', '11', '12', '13',
           '20', '21', '22', '23',
           '30', '31', '32', '33',
           '40', '41', '42', '43',
           '50', '51', '52', '53']
rep_list = [replicate for replicate in expt.replicate_trials if
            replicate.trial_identifier.strain.name[:2] in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
analyte_of_interest = 'GFP'
attr = 'data_vector'
rbs_mean = {}
rbs_std = {}
deg_mean = {}
deg_std = {}
#First separate by Degradation Tag - Observe effect of RBS
trace_list = []
for index,rbs_level in enumerate(['1','2','3','4','5']):
    temp_reps = [rep for rep in rep_list if rep.trial_identifier.strain.name[0]==rbs_level]
    temp_reps = sorted(temp_reps, key=lambda rep:str(rep.trial_identifier.strain.name))
    max_ss = max([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in temp_reps])    
    deg_mean = {**deg_mean, **{rep.trial_identifier.strain.name[:2]:
                               1/(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)
                               for rep in temp_reps}}
    deg_std = {**deg_std, **{rep.trial_identifier.strain.name[:2]:
                               (getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)/(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)**2
                               for rep in temp_reps}}
for index,deg_level in enumerate(['0','1','2','3']):
    temp_reps = [rep for rep in rep_list if rep.trial_identifier.strain.name[1]==deg_level]
    temp_reps = sorted(temp_reps, key=lambda rep:str(rep.trial_identifier.strain.name))
    min_ss = min([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in temp_reps])
    
    
    rbs_mean = {**rbs_mean, **{rep.trial_identifier.strain.name[:2]:
                               (getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/min_ss)
                               for rep in temp_reps}}
    rbs_std = {**rbs_std, **{rep.trial_identifier.strain.name[:2]:
                               (getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]/min_ss)
                               for rep in temp_reps}}
strains = ['10', '11', '12', '13',
           '20', '21', '22', '23',
           '30', '31', '32', '33',
           '40', '41', '42', '43',
           '50', '51', '52', '53']
rbs_mean_list = [rbs_mean[strain] for strain in strains]
rbs_std_list = [rbs_std[strain] for strain in strains]
deg_mean_list = [deg_mean[strain] for strain in strains]
deg_std_list = [deg_std[strain] for strain in strains]

gfp_df = pd.DataFrame(data={'rbs_mean': rbs_mean_list,
                             'rbs_std': rbs_std_list,
                             'deg_mean': deg_mean_list,
                             'deg_std': deg_std_list,
                             'lac_code':strains})
gfp_df = gfp_df.set_index('lac_code')
display(gfp_df)

,rbs_mean,rbs_std,deg_mean,deg_std
lac_code,,,,
10,1.000000,0.278764,1.000000,0.278764
11,1.000000,0.261222,1.973846,0.515612
12,1.000000,0.543878,1.867842,1.015878
13,1.000000,0.414014,1.820719,0.753803
20,3.686672,0.152844,1.000000,0.041459
21,2.829060,0.713982,2.572205,0.649159
22,3.225170,0.235452,2.135119,0.155873
23,2.020498,0.307852,3.322148,0.506177
30,9.795271,1.183984,1.021299,0.123447


In [29]:
gfp_df.to_excel('C:\\Users\\User\\OneDrive - University of Toronto\\Research\\PhD_UofT\\Data\\Toggle_Switch_Project\\Experiments\\Screening\\pTSFusXXXX\\Speed\\gfp_df.xlsx')

In [7]:
analyte_of_interest = 'mCherry'
strains = ['0000']
trace_list = []
temp_colors=[colors['blacks'][4]]
rep_list = [replicate for replicate in expt_atc.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
for i,strain in enumerate(strains):
    x = ['Reference Strains']
    replicate = [rep for rep in rep_list if rep.trial_identifier.strain.name==strain]
    if replicate:
        replicate = replicate[0]
        y = [(getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1])]
        y_errs = [np.sqrt(getattr(replicate.std.analyte_dict[analyte_of_interest], attr)[-1]**2)]


        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=temp_colors[i], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(strain),
                             name=str(strain)))    
width=300
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=16, color='black'),
                   legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=16, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
fig1['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig1)

In [8]:

strains = ['0000']
reps = [rep for expt in [expt_atc, expt_iptg] for rep in expt.replicate_trials if rep.trial_identifier.strain.name=='0000']
rep_atc, rep_iptg = reps
trace_list = []
attr = 'od_normalized_data'
x = ['ATC', 'IPTG']
y = [(getattr(rep.avg.analyte_dict['GFP'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_errs = [(getattr(rep.std.analyte_dict['GFP'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
trace_list.append(go.Bar(x=x,
                     y=y, yaxis='y1',
                     error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                     marker=dict(color=colors['greens'][3], line=dict(width=0)), offsetgroup=1,
                     showlegend=False,# yaxis='y2',
                     legendgroup=str(strain),
                     name=str(strain)))  

y = [(getattr(rep.avg.analyte_dict['mCherry'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_errs = [(getattr(rep.std.analyte_dict['mCherry'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
trace_list.append(go.Bar(x=x,
                     y=y, yaxis='y2',
                     error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                     marker=dict(color=colors['reds'][3], line=dict(width=0)),offsetgroup=2,
                     showlegend=False,# yaxis='y2',
                     legendgroup=str(strain),
                     name=str(strain)))  
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=400, width=350, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=16, color='black'),
                   legend_x=0.02, bargap=0.5, bargroupgap=0.2, title=title,
                   legend_font=dict(family='Myriad Pro', size=16, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
fig1['layout']['yaxis1'] = dict(title='GFP Fluorescence (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=6, tickangle=0, type='linear', range=(0, 5200),
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
fig1['layout']['yaxis2'] = dict(title='mCherry Fluorescence(a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'), range=(0,1300), dtick=250,
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=6, tickangle=0, type='linear', anchor='x',overlaying='y1',
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='right')


plot(fig1)
fig1.write_image('bistability.svg', format='svg')

In [134]:
rep_iptg.avg.analyte_dict['GFP'].pd_series[0]/rep_iptg.avg.analyte_dict['mCherry'].pd_series[0]

0.5825361229758641

In [135]:
rep_atc.avg.analyte_dict['GFP'].pd_series[0]/rep_atc.avg.analyte_dict['mCherry'].pd_series[0]

28.712544064155896

In [82]:
reps = [rep for expt in [expt_atc, expt_iptg] for rep in expt.replicate_trials if rep.trial_identifier.strain.name=='0000']
rep_atc, rep_iptg = reps

In [70]:
for rep in reps:
    print(rep.trial_identifier)

strain: 0000,	media: RDM,	env: None 
strain: 0000,	media: RDM,	env: None 


In [63]:
rep[0].avg.analyte_dict['GFP'].od_normalized_data

array([629.18020683])

In [88]:
gfp_max = rep_atc.avg.analyte_dict['GFP'].od_normalized_data[0]
gfp_err = rep_atc.std.analyte_dict['GFP'].od_normalized_data[0]
mche_max = rep_iptg.avg.analyte_dict['mCherry'].od_normalized_data[0]
mche_err = rep_iptg.std.analyte_dict['mCherry'].od_normalized_data[0]

In [124]:
for  i in range(2):
    print(i)

0
1


In [141]:
strains = ['0000']
reps = [rep for expt in [expt_atc, expt_iptg] for rep in expt.replicate_trials if rep.trial_identifier.strain.name=='0000']
rep_atc, rep_iptg = reps
trace_list = []
attr = 'od_normalized_data'
x = ['ATC', 'IPTG']
y = [(getattr(rep.avg.analyte_dict['GFP'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_scaled = [val/gfp_max for val in y]
y_errs = [(getattr(rep.std.analyte_dict['GFP'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_errs_scaled = [y_scaled[i]*(np.sqrt(y_errs[i]**2/y[i]**2 + gfp_err**2/gfp_max**2)) for i in range(2)]
trace_list.append(go.Bar(x=x,
                     y=y_scaled, yaxis='y1',
                     error_y=dict(type='data', array=y_errs_scaled, thickness=1.5, width=3, color='black'),
                         marker=dict(color=colors['greens'][3], line=dict(width=0)), offsetgroup=1,
                     showlegend=False,# yaxis='y2',
                     legendgroup=str(strain),
                     name=str(strain)))  

y = [(getattr(rep.avg.analyte_dict['mCherry'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_scaled = [val/mche_max for val in y]
y_errs = [(getattr(rep.std.analyte_dict['mCherry'], attr)[-1]) for rep in [rep_atc, rep_iptg]]
y_errs_scaled = [y_scaled[i]*(np.sqrt(y_errs[i]**2/y[i]**2 + mche_err**2/mche_max**2)) for i in range(2)]

trace_list.append(go.Bar(x=x,
                     y=y_scaled, yaxis='y1',
                     error_y=dict(type='data', array=y_errs_scaled, thickness=1.5, width=3, color='black'),
                     marker=dict(color=colors['reds'][3], line=dict(width=0)),offsetgroup=2,
                     showlegend=False,# yaxis='y2',
                     legendgroup=str(strain),
                     name=str(strain)))  
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=400, width=350, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=16, color='black'),
                   legend_x=0.02, bargap=0.5, bargroupgap=0.2, title=title,
                   legend_font=dict(family='Myriad Pro', size=16, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
fig1['layout']['yaxis1'] = dict(title='Fluorescence Normalized to Max (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=6, tickangle=0, type='linear', range=(0, 1.25),
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')



plot(fig1)